In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
output_notebook()

from scripts_viz.visualization_utils import *
from scripts_viz.visualization_perfgrid import *
from scripts_viz.visualization_utils import TTQcolor
from scripts_mlflow.mlflow_utils import *

from bokeh.io import export_png
from bokeh.layouts import gridplot, row, column

import mlflow


Loading BokehJS ...

In [2]:
pd.set_option("display.max_rows", 101)

In [3]:
expnames = set([exp.name for exp in mlflow.tracking.MlflowClient().list_experiments()])
expnames

{'MLP_enriched_time_seq24000_6000_imp',
 'MLP_enriched_time_seq24000_6000_p180',
 'MLP_enriched_time_seq24000_6000_p90',
 'MLP_enriched_time_seq26000_2000_opt_p180',
 'benchmarks_shuffle_imp',
 'benchmarks_shuffle_p180',
 'benchmarks_shuffle_p90',
 'benchmarks_time_old_imp',
 'benchmarks_time_old_p180',
 'benchmarks_time_old_p90',
 'benchmarks_time_opt_imp',
 'benchmarks_time_opt_p90',
 'enriched_shuffle_imp',
 'enriched_shuffle_p180',
 'enriched_shuffle_p90',
 'enriched_time_seq24000_6000_imp',
 'enriched_time_seq24000_6000_opt_imp',
 'enriched_time_seq26000_2000_opt_p180',
 'enriched_time_seq26000_2000_opt_p90',
 'enriched_time_seq26000_2000_p180',
 'enriched_time_seq26000_2000_p90'}

In [4]:
TTQcolor.keys()

dict_keys(['font', 'lightGrey', 'borderColour', 'cream', 'lightCream', 'background', 'link', 'brightLink', 'marketplaceOrange', 'warningRed', 'Salmon', 'victorian', 'whiteGrey', 'eightyGrey', 'blueGrey', 'sixtyGrey', 'navy', 'darkPurple', 'darkNavy', 'darkCyan', 'redBrown', 'richBrown', 'algae', 'mutedBlue', 'azureBlue', 'electric', 'sky', 'turq', 'pea', 'ocean', 'richOrange', 'richPeach', 'yell', 'yellowOrange', 'peach', 'bloodRed', 'PPTbg'])

In [5]:
colors = [ TTQcolor['bloodRed'], TTQcolor['pea']]

#--------------------------------------------------

sgd_cols = ['eta0', 'shuffle', 'fit_intercept', 'n_iter_no_change', 'penalty', 'max_iter', 'alpha', 'early_stopping',
           'learning_rate', 'loss', 'val_auc', 'test_auc']

sgd_spider_cols = ['eta0','max_iter','alpha','test_auc', 'val_auc']

sgd_spider_cols_to_norm = ['eta0','max_iter','alpha']

#--------------------------------------------------
rf_cols = ['n_estimators', 'max_depth', 'max_features', 'max_leaf_nodes', 
                                        'min_samples_leaf', 'min_samples_split', 'bootstrap',
                                        'criterion', 'val_auc', 'test_auc']
rf_spider_cols = ['n_estimators',  'max_leaf_nodes', 'min_samples_leaf', 'min_samples_split', 'test_auc', 'val_auc']

rf_spider_cols_to_norm = ['n_estimators',  'max_leaf_nodes', 'min_samples_leaf', 'min_samples_split']

#--------------------------------------------------
ann_cols = ['batch_size', 'optimizer', 'epochs_actual', 'class_1_weight', 'early_stopping', 'early_stopping_metric',
            'hidden_layers_no', 'hidden_nodes', 'hl_out_activations', 'loss_func', 'dropout', 'tr_accuracy',
             'val_auc', 'test_auc']


In [6]:
#vals_1 = [v for v in viz_1.loc['test_predictions', 'benchmarks_shuffle_imp_SGD_190824_154114'].split(',')[:-1]]
#vals_1 = [float(i) for i in vals_1]
#vals_2 = [v for v in viz_1.loc['test_predictions', 'benchmarks_shuffle_imp_RF_190824_154156'].split(',')[:-1]]
#vals_2 = [float(i) for i in vals_2]
#a = cor_scatterplot(vals_1, vals_2, xaxis_label='benchmarks_shuffle_imp_SGD_190824_154114', 
              #  yaxis_label='benchmarks_shuffle_imp_RF_190824_154156', circle_size=10, title_size='12pt')

#show(a)

In [7]:
for clf in ('rf_', 'sgd_'):
    for mult_exp in [['enriched_time_seq24000_6000_opt_imp', 'benchmarks_time_opt_imp']]:
                      #('benchmarks_shuffle_p90', 'benchmarks_time_old_p90'),
                      #('benchmarks_shuffle_p180', 'benchmarks_time_old_p180')
                      #]:
        
        viz_list=[]
        for experiment in mult_exp:
            viz_tmp = create_exp_df(experiment)
            viz_tmp.columns = experiment+'_'+viz_tmp.columns
            viz_list.append(viz_tmp)
            
        viz = pd.concat(viz_list, axis=1)
        #experiment_1 = double_exp[0]
        #experiment_2 = double_exp[1]
        
        #viz_1 = create_exp_df(experiment_1)
        #viz_1.columns = experiment_1+'_'+viz_1.columns
        #viz_2 = create_exp_df(experiment_2)
        #viz_2.columns = experiment_2+'_'+viz_2.columns
        #viz = pd.concat([viz_1, viz_2], axis=1)
        
        if clf=='rf_':
            model_filter = ['RandomForestClassifier']
            metric_cols = rf_cols
            spider_cols = rf_spider_cols
            spider_cols_to_norm = rf_spider_cols_to_norm
            index_header = 'RF'
            spr_settings = {'viz_dict': viz, 
                                             'metric_list': rf_cols, 
                                             'model_type': ['RandomForestClassifier'], 
                                             'color_cells':True, 
                                             'colors':colors,
                                             'index_header':'RF', 
                                             'width':1200, 
                                             'height':120, 
                                             'index_width':40,
                                             'row_height':25}

        elif clf=='sgd_':
            model_filter = ['SGDClassifier']
            metric_cols = sgd_cols
            spider_cols = sgd_spider_cols
            spider_cols_to_norm = sgd_spider_cols_to_norm
            index_header = 'SGD'
            spr_settings = {'viz_dict': viz, 
                                             'metric_list': sgd_cols, 
                                             'model_type': ['SGDClassifier'], 
                                             'color_cells':True, 
                                             'colors':colors,
                                             'index_header':'SGD', 
                                             'width':1200, 
                                             'height':120, 
                                             'index_width':40,
                                             'row_height':25}
            
        grid = performance_grid(viz,
                     model_filter = model_filter, 
                     legend_font_size='12pt', 
                     fpr_font_size='9pt', 
                     bestFprOnly=True, 
                     rocs_p_width=600, 
                     rocs_p_height=600,
                     rocs_line_width=2, 
                     single_spider=True, 
                     add_spider=True,
                     spider_in_row=2, 
                     spiders_params = spider_cols,
                     spiders_params_to_norm=spider_cols_to_norm,
                     spider_p_width=600, 
                     spider_p_height=600, 
                     spider_text_size='8pt', 
                     spider_line_width=4.5, 
                     spider_fill_alpha=0.1,
                     spider_margin_distance=0.25, 
                     normalize_spider=True,
                     single_row_folds=True, 
                    add_corr_scatter=False,
                     folds_p_width=600, 
                     folds_p_height=600, 
                     folds_xlabelorientation=1.55, 
                     folds_group_text_font_size='8pt',
                     folds_in_row=2, 
                     spreadsheet_settings = [spr_settings],
                     plot_feat_importance=True,
                     normalize_importance=False,
                     fimp_text_group_size = '10pt',
                     colors=colors)
        
        name = clf
        for e in range(len(mult_exp),2):
            name+=experiment
        name+='.png'
        
        
        #export_png(grid, name)

C:\ProgramData\Anaconda3\envs\thesis_mlflow\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [8]:
viz

,enriched_time_seq24000_6000_opt_imp_RF_190826_82322,enriched_time_seq24000_6000_opt_imp_SGD_190826_8232,benchmarks_time_opt_imp_SGD_190826_81513,benchmarks_time_opt_imp_RF_190826_81525
alpha,NaN,0.0001,0.0001,NaN
artifact_uri,file:///C:/Users/david/Dropbox/University/MscD...,file:///C:/Users/david/Dropbox/University/MscD...,file:///C:/Users/david/Dropbox/University/MscD...,file:///C:/Users/david/Dropbox/University/MscD...
average,NaN,False,False,NaN
bootstrap,True,NaN,NaN,True
class_weight,balanced,None,None,balanced
criterion,gini,NaN,NaN,gini
early_stopping,NaN,False,False,NaN
end_time,2019-08-26 07:23:08.309000,2019-08-26 07:23:08.309000,2019-08-26 07:15:29.648000,2019-08-26 07:15:29.648000
epsilon,NaN,0.1,0.1,NaN
eta0,NaN,0.01,0.001,NaN


In [9]:
show(grid)